In [ ]:
import json
import numpy as np
def load_json():
    
    authors = []
    features = []
    scores = []
    
    with open('dataset.json') as data_file:
        data = json.load(data_file)
        labels = data["column_descriptors"]
        author_data = data["author_data"]
        
        for author in author_data.keys():
            authors.append(author)
            features.append(author_data[author]['feature_vecs'])
            scores_str = author_data[author]['scores']
            scores.append(np.asarray([int(score_) for score_ in scores_str]))
    
    # features to numpy array
    features = [np.asarray(feature) for feature in features]
    
    return labels, authors, features, scores

In [ ]:
labels, authors, features, scores = load_json()
print (authors[0], features[0].shape, scores[0].shape)
print (labels)

In [ ]:
# training and test split
import random
cnt_train = 0.7 * len(authors) + 1

print (cnt_train)

train_authors_indices = set()

while len(train_authors_indices) < cnt_train:
    train_authors_indices.add(random.randint(0, len(authors) - 1))
    
test_authors_indices = set()

for i in range(len(authors)):
    if i not in train_authors_indices:
        test_authors_indices.add(i)

train_authors = [authors[i] for i in train_authors_indices]
        
test_authors = [authors[i] for i in test_authors_indices]

print(len(train_authors))
print(len(test_authors))

In [ ]:
X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

for index in train_authors_indices:
    X_train_list.extend(features[index])
    y_train_list.extend(scores[index])
    
for index in test_authors_indices:
    X_test_list.extend(features[index])
    y_test_list.extend(scores[index])

X_train = np.asarray(X_train_list)
y_train = np.asarray(y_train_list)
X_test = np.asarray(X_test_list)
y_test = np.asarray(y_test_list)

print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
# y is vector of labels
def create_labels(y):
    y_l = np.copy(y)
    for i in range(y.shape[0]):
        if y[i] >= 80: 
            y_l[i] = 5
        if y[i] >= 60 and y[i] < 80: 
            y_l[i] = 4
        if y[i] >= 40 and y[i] < 60:
            y_l[i] = 3
        if y[i] >= 20 and y[i] < 40:
            y_l[i] = 2
        if y[i] < 20: 
            y_l[i] = 1
    return y_l

In [ ]:
y_train_labeled = create_labels(y_train)
y_test_labeled = create_labels(y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_features='log2')
rfc.fit(X_train, y_train)

print (rfc.score(X_test, y_test))

In [ ]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train, y_train) 

print (clf.score(X_test, y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

print (clf.score(X_test, y_test))

In [ ]:
from sklearn.model_selection import GridSearchCV
import multiprocessing
cores=multiprocessing.cpu_count()-2

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100, 1000]}
svc = SVC()
clf = GridSearchCV(svc, parameters, n_jobs=cores, verbose=2)
clf.fit(X_train, y_train)
print (clf.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
print (gbc.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier()
abc.fit(X_train, y_train)
print (abc.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators=30)
etc.fit(X_train, y_train)
print (etc.score(X_test, y_test))

In [ ]:
print X.shape, y.reshape((y.shape[0], 1)).shape

In [ ]:
#dataset = X#np.concatenate((X, y.T), axis=1)

dataset = np.concatenate((X,y[:,None]),axis=1)
#print X.shape[0] == y.shape[0]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
df = pd.DataFrame(dataset)
#sns.pairplot(df)
#plt.show()